In [2]:
import requests
from bs4 import BeautifulSoup
import json
import time
import re # Regular expressions

# --- Configuratie ---
BASE_URL = "https://www.zorgkaartnederland.nl"
TARGET_REGION = "nijmegen" # OF "arnhem"

if TARGET_REGION == "arnhem":
    START_PATH = "/verpleeghuis-en-verzorgingshuis/arnhem"
    OUTPUT_FILE = "zorginstellingen_arnhem_w_beds_pc.json" # Aangepaste output naam
    REGION = "Arnhem"
elif TARGET_REGION == "nijmegen":
    START_PATH = "/verpleeghuis-en-verzorgingshuis/nijmegen"
    OUTPUT_FILE = "zorginstellingen_nijmegen_w_beds_pc.json" # Aangepaste output naam
    REGION = "Nijmegen"
else:
    print(f"Fout: Onbekende TARGET_REGION '{TARGET_REGION}'. Kies 'arnhem' of 'nijmegen'.")
    exit()

PARAMS = "?d=25"
MAX_PAGES = 10
REQUEST_TIMEOUT = 15
PAUSE_BETWEEN_PAGES = 2
PAUSE_BETWEEN_ITEMS = 0.7

HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'Accept-Language': 'nl-NL,nl;q=0.9,en-US;q=0.8,en;q=0.7',
    'Referer': BASE_URL + '/'
}

# --- CSS Selectors ---
selectors = {
    "result_item": "div.filter-result",
    "name": "a.filter-result__name",
    "places": "div.filter-result__places",
    "distance": "div.filter-result__places span.badge",
    "rating_score": "div.filter-result__score",
    "rating_count_container": "div.filter-result__body__right p",
    "facilities_link": "a.tab-link:contains('Faciliteiten')",
    "facilities_table": "table.table-no-borders",
    "facilities_table_row": "tr.row.mx-0"
}
# ------------------------------------------------------------------------------------

def parse_rating_count(text):
    """Probeert het aantal waarderingen uit een tekst te halen."""
    if not text: return None
    match = re.search(r'\d+', text)
    return int(match.group(0)) if match else None

def get_aantal_plaatsen_totaal(item_url, session):
    """
    Haalt het totaal aantal plaatsen EN de postcode op van de detail/faciliteitenpagina.
    Retourneert een tuple (count, postcode), waarbij elk None kan zijn.
    """
    postcode = None # Initialize postcode variable
    count = None    # Initialize count variable

    try:
        # 1. Haal detailpagina op
        print(f"  -> Detailpagina ophalen: {item_url}")
        detail_response = session.get(item_url, timeout=REQUEST_TIMEOUT)
        detail_response.raise_for_status()
        detail_soup = BeautifulSoup(detail_response.text, 'html.parser')

        # === NIEUW: Extraheer Postcode uit JSON-LD ===
        script_tag = detail_soup.find('script', type='application/ld+json')
        if script_tag and script_tag.string:
            try:
                ld_json_data = json.loads(script_tag.string)
                address_data = ld_json_data.get('address', {}) # Gebruik .get met default {}
                postcode = address_data.get('postalCode') # Geeft None terug als key niet bestaat
                if postcode:
                    print(f"  -> Gevonden postcode (JSON-LD): {postcode}")
                else:
                    print(f"  -> Waarschuwing: 'postalCode' niet gevonden in JSON-LD adres op {item_url}")
            except json.JSONDecodeError as json_err:
                print(f"  -> Fout: Kon JSON-LD niet parsen op {item_url}: {json_err}")
            except Exception as e:
                 print(f"  -> Fout: Onverwachte fout bij verwerken JSON-LD op {item_url}: {e}")
        else:
            print(f"  -> Waarschuwing: JSON-LD script tag niet gevonden op {item_url}")
        # === EINDE: Extractie Postcode ===


        # 2. Zoek 'Faciliteiten' link
        faciliteiten_link = detail_soup.find(
            'a',
            class_='tab-link',
            string=lambda t: t and 'Faciliteiten' in t.strip()
        )

        if not faciliteiten_link or not faciliteiten_link.has_attr('href'):
            print(f"  -> 'Faciliteiten' link niet gevonden op {item_url}. Kan aantal plaatsen niet ophalen.")
            return count, postcode # <<< Retourneer (None, mogelijk gevonden postcode)

        faciliteiten_path = faciliteiten_link['href']
        if faciliteiten_path.startswith('/'):
            faciliteiten_url = BASE_URL + faciliteiten_path
        elif not faciliteiten_path.startswith(('http:', 'https:')):
             print(f"  -> Waarschuwing: onverwacht formaat faciliteiten href: {faciliteiten_path}")
             faciliteiten_url = BASE_URL + "/" + faciliteiten_path.lstrip('/')
        else:
             faciliteiten_url = faciliteiten_path

        # 3. Haal faciliteitenpagina op
        print(f"  -> Faciliteitenpagina ophalen: {faciliteiten_url}")
        faciliteiten_response = session.get(faciliteiten_url, timeout=REQUEST_TIMEOUT)
        faciliteiten_response.raise_for_status()
        faciliteiten_soup = BeautifulSoup(faciliteiten_response.text, 'html.parser')

        # 4. Zoek de tabel(len) en de specifieke rij voor aantal plaatsen
        target_tables = faciliteiten_soup.select(selectors["facilities_table"])
        if not target_tables:
            print(f"  -> Geen tabel(len) met class 'table-no-borders' gevonden op faciliteitenpagina: {faciliteiten_url}")
            return count, postcode # <<< Retourneer (None, mogelijk gevonden postcode)

        found_count = False
        for target_table in target_tables:
             heading = target_table.find_previous_sibling('h3')
             if heading and "Aantal plekken en wachttijd" in heading.get_text(strip=True):
                  print(f"  -> Correcte tabel gevonden ('Aantal plekken en wachttijd').")
                  rows = target_table.select(selectors["facilities_table_row"])
                  for row in rows:
                      header = row.select_one('th.col-6')
                      value_cell = row.select_one('td.col-6')

                      if header and value_cell and "Aantal plaatsen totaal" == header.get_text(strip=True):
                          try:
                              count = int(value_cell.get_text(strip=True)) # Update count variabele
                              found_count = True
                              print(f"  -> Gevonden aantal plaatsen totaal: {count}")
                              # Ga NIET meteen returnen, controleer eerst alle tabellen
                              break # Stop met zoeken in deze tabel
                          except ValueError:
                              print(f"  -> Fout: Kon waarde '{value_cell.get_text(strip=True)}' niet converteren naar getal.")
                              count = None # Fout bij conversie, zet count op None
                              found_count = True # Markeer dat we de rij wel vonden
                              break # Stop met zoeken in deze tabel
                  if found_count:
                        break # Stop met zoeken door tabellen als we de rij hebben verwerkt
        # Einde tabel loop

        if not found_count:
             print(f"  -> Rij 'Aantal plaatsen totaal' niet gevonden in de tabel(len) 'Aantal plekken en wachttijd'.")
             # Count blijft None als het niet gevonden is

        # Retourneer de gevonden (of None) waarden aan het einde
        return count, postcode

    except requests.exceptions.Timeout:
        print(f"  -> Fout: Timeout bij ophalen van {item_url} of faciliteitenpagina.")
        return count, postcode # Retourneer wat we eventueel al hadden
    except requests.exceptions.RequestException as e:
        print(f"  -> Fout: Netwerkfout tijdens ophalen details/faciliteiten voor {item_url}: {e}")
        return count, postcode # Retourneer wat we eventueel al hadden
    except Exception as e:
        print(f"  -> Fout: Onverwachte fout bij verwerken {item_url}: {e}")
        return None, None # In dit geval weten we niks zeker

def scrape_zorgkaart():
    all_data = []
    session = requests.Session()
    session.headers.update(HEADERS)

    print(f"Starten met scrapen van ZorgkaartNederland voor regio: {REGION}...")
    print(f"Output wordt opgeslagen in: {OUTPUT_FILE}")
    print(f"Pauze tussen pagina's: {PAUSE_BETWEEN_PAGES}s, Pauze tussen items: {PAUSE_BETWEEN_ITEMS}s")

    for page_num in range(1, MAX_PAGES + 1):
        print(f"\n--- Scraping pagina {page_num} voor {REGION} ---")

        if page_num == 1:
            page_path = START_PATH + PARAMS
        else:
            page_path = f"{START_PATH}/pagina{page_num}{PARAMS}"

        current_url = BASE_URL + page_path
        print(f"Pagina URL: {current_url}")

        try:
            response = session.get(current_url, timeout=REQUEST_TIMEOUT)
            response.raise_for_status()

            soup = BeautifulSoup(response.text, 'html.parser')
            results = soup.select(selectors["result_item"])

            if not results:
                print(f"Geen resultaten (items) meer gevonden op pagina {page_num}. Stoppen met pagineren.")
                break

            print(f"Aantal resultaten gevonden op deze pagina: {len(results)}")

            for index, item in enumerate(results):
                print(f"\nVerwerken item {index + 1}/{len(results)} op pagina {page_num}...")
                data = {"Regio": REGION}

                # --- Basisgegevens extractie ---
                name_element = item.select_one(selectors["name"])
                data["Naam"] = name_element.get_text(strip=True) if name_element else None
                item_url = BASE_URL + name_element['href'] if name_element and name_element.has_attr('href') else None
                data["Url"] = item_url

                distance_element = item.select_one(selectors["distance"])
                distance_text = distance_element.get_text(strip=True) if distance_element else None
                data["Afstand"] = distance_text

                places_element = item.select_one(selectors["places"])
                if places_element:
                    full_places_text = places_element.get_text(strip=True)
                    location_text = full_places_text.replace(distance_text, '').strip() if distance_text else full_places_text
                    data["Locatie"] = re.sub(r'\s+', ' ', location_text).strip('() ')
                else:
                    data["Locatie"] = None

                rating_score_element = item.select_one(selectors["rating_score"])
                data["Cijfer"] = None
                if rating_score_element:
                    rating_text = rating_score_element.get_text(strip=True).replace(',', '.')
                    try:
                        data["Cijfer"] = float(rating_text)
                    except ValueError:
                        print(f"  - Waarschuwing: Kon cijfer '{rating_text}' niet converteren voor {data.get('Naam')}.")
                        pass

                rating_count_container = item.select_one(selectors["rating_count_container"])
                rating_count_text = rating_count_container.get_text(strip=True) if rating_count_container else None
                data["Aantal Waarderingen"] = parse_rating_count(rating_count_text)

                # --- Haal aantal plaatsen totaal EN postcode op ---
                data["Aantal Plaatsen Totaal"] = None # Standaardwaarde
                data["Postcode"] = None              # Standaardwaarde voor postcode

                if item_url:
                    # === AANGEPAST: Unpack het tuple ===
                    totaal_count, postcode_val = get_aantal_plaatsen_totaal(item_url, session)
                    data["Aantal Plaatsen Totaal"] = totaal_count
                    data["Postcode"] = postcode_val # Wijs de mogelijk gevonden postcode toe
                else:
                    print(f"  - Waarschuwing: Geen URL gevonden voor item, kan details niet ophalen.")


                # --- Voeg data toe aan lijst ---
                if data["Naam"]:
                     # === AANGEPAST: Log message met postcode ===
                    print(f"  -> Data verzameld voor: {data['Naam']} (Plaatsen: {data.get('Aantal Plaatsen Totaal', 'N/B')}, Postcode: {data.get('Postcode', 'N/B')})")
                    all_data.append(data)
                else:
                    print(f"  - Waarschuwing: Item overgeslagen op pagina {page_num} wegens ontbrekende naam.")

                time.sleep(PAUSE_BETWEEN_ITEMS) # Pauze

            if page_num < MAX_PAGES:
                print(f"\nEinde pagina {page_num}, pauzeren ({PAUSE_BETWEEN_PAGES}s)...")
                time.sleep(PAUSE_BETWEEN_PAGES)

        except requests.exceptions.Timeout:
            print(f"Fout: Timeout bij ophalen van hoofdpagina {current_url}. Stoppen.")
            break
        except requests.exceptions.RequestException as e:
            print(f"Fout: Netwerkfout bij ophalen {current_url}: {e}. Stoppen.")
            break
        except Exception as e:
            print(f"Fout: Onverwachte fout tijdens parsen pagina {page_num}: {e}")
            print("Proberen door te gaan naar volgende pagina (indien mogelijk)...")
            time.sleep(PAUSE_BETWEEN_PAGES)
            continue

    return all_data

if __name__ == "__main__":
    try:
        import requests
        from bs4 import BeautifulSoup
    except ImportError:
        print("="*60); print("FOUT: Libraries 'requests' en/of 'beautifulsoup4' niet geïnstalleerd."); print("Installeer ze met: pip install requests beautifulsoup4"); print("="*60); exit(1)

    start_time = time.time()
    scraped_data = scrape_zorgkaart()
    end_time = time.time()

    print("\n" + "="*60); print(f"Scrapen voltooid in {end_time - start_time:.2f} seconden."); print(f"Totaal {len(scraped_data)} instellingen gevonden en verwerkt voor {REGION}."); print("="*60)

    if scraped_data:
        try:
            with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
                json.dump(scraped_data, f, ensure_ascii=False, indent=4)
            print(f"\nData succesvol opgeslagen in bestand: {OUTPUT_FILE}")
        except IOError as e:
            print(f"\nFout: Kon niet schrijven naar bestand {OUTPUT_FILE}: {e}")
        except Exception as e:
            print(f"\nFout: Onverwachte fout bij opslaan JSON: {e}")
    else:
        print("\nGeen data gescraped. Controleer selectors, URL, website beschikbaarheid en foutmeldingen hierboven.")

Starten met scrapen van ZorgkaartNederland voor regio: Nijmegen...
Output wordt opgeslagen in: zorginstellingen_nijmegen_w_beds_pc.json
Pauze tussen pagina's: 2s, Pauze tussen items: 0.7s

--- Scraping pagina 1 voor Nijmegen ---
Pagina URL: https://www.zorgkaartnederland.nl/verpleeghuis-en-verzorgingshuis/nijmegen?d=25
Aantal resultaten gevonden op deze pagina: 20

Verwerken item 1/20 op pagina 1...
  -> Detailpagina ophalen: https://www.zorgkaartnederland.nl/zorginstelling/verpleeghuis-en-verzorgingshuis-rijnwaal-zorggroep-locatie-sint-jozef-lent-lent-12650
  -> Gevonden postcode (JSON-LD): 6663 CP
  -> Faciliteitenpagina ophalen: https://www.zorgkaartnederland.nl/zorginstelling/verpleeghuis-en-verzorgingshuis-rijnwaal-zorggroep-locatie-sint-jozef-lent-lent-12650/faciliteiten
  -> Rij 'Aantal plaatsen totaal' niet gevonden in de tabel(len) 'Aantal plekken en wachttijd'.
  -> Data verzameld voor: RijnWaal Zorggroep, locatie Sint Jozef Lent (Plaatsen: None, Postcode: 6663 CP)

Verwerken